In [1]:
from src.utils.FeatureConfig import FeatureConfig
from src.utils.FeatureExtractor import FeatureExtractor

In [2]:
config = FeatureConfig(pathToDataset='data/fakeNewsDatasets/fakeNewsDataset/',
                       pathToGI='inquirerbasic.xls',
                       collectPosFromCorpus=True,
                       collectProductionFromCorpus=True,
                       )

In [3]:
featureExtractor = FeatureExtractor(config)

/home/allen/anaconda3/envs/syntax/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:391: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['far', 'make', 'u'] not in stop_words.
  'stop_words.' % sorted(inconsistent))
2021-03-16 11:18:33 INFO: Writing properties to tmp file: corenlp_server-0d847a2c14514697.props
2021-03-16 11:18:33 INFO: Starting server with command: java -Xmx16G -cp /home/allen/stanza_corenlp/* edu.stanford.nlp.pipeline.StanfordCoreNLPServer -port 9000 -timeout 30000 -threads 5 -maxCharLength 100000 -quiet False -serverProperties corenlp_server-0d847a2c14514697.props -annotators tokenize,ssplit,pos,lemma,ner,parse -preload -outputFormat serialized


In [5]:
featureExtractor.save("save/state.pkl")

In [ ]:
import os

def loadDataset(path):
    data = []
    files = os.listdir(path)
    for file in files:
        with open(os.path.join(path, file)) as f:
            data.append(f.read())
    return data

def dataToFeatures(data, featureExtractor):
    
    features = []
    for doc in data:
        features.append(featureExtractor.extract())
legitData = loadDataset('data/fakeNewsDatasets/fakeNewsDataset/legit/')
fakeData = loadDataset('data/fakeNewsDatasets/fakeNewsDataset/fake/')

In [ ]:
legitFeatures = featureExtractor.extractSentences(legitData)
fakeFeatures = featureExtractor.extractSentences(fakeData)

In [ ]:
featureExtractor.extractorName()

In [ ]:
import numpy as np

np.array([np.array([1,2]), np.array([13,23])])

In [ ]:
np.concatenate([np.zeros((4,4)), np.ones((4,4))], axis = 1)